In [1]:
import networkx as nx
import seaborn as sns
import pandas as pd
import glob
import ast

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
filepath = r"C:\Users\Alessandro\snap_github\Fake-news\Analisi 2" # use your path
all_files = glob.glob(filepath + "/*.txt")
G = {}
j = 0

for filename in all_files:
    G[j]=nx.DiGraph()
    fh=open(filename,'r')
    for i, line in enumerate(fh):
        s=line.strip()
        s=s.strip(";").split()
        if (i>6) and (s[0] != 'label') and (s[0] != '}') and (s[1] != '[style=filled,'):
            origin=int(s[0])
            dest=int(s[2])
            G[j].add_edge(origin,dest)
    fh.close()
    j += 1

In [3]:
for i in range (len(G)):
    print('The network has', len(G[i]), 'nodes and', len(G[i].edges()), 'links.')

The network has 43361 nodes and 56742 links.


In [4]:
for i in range (len(G)):
    print("Is the graph simply connected?", nx.is_strongly_connected(G[i]))

Is the graph simply connected? False


### Show the components of the graph

In [5]:
for i in range (len(G)):
    print("The graph has", nx.number_strongly_connected_components(G[i]),"connected components")

The graph has 43360 connected components


### Extract the largest Connected Component as a subgraph

In [6]:
len(G)

1

In [7]:
graphs = {}

for i in range (len(G)):
    graphs[i] = []
    for c in nx.strongly_connected_components(G[i]):
        graphs[i].append(G[i].subgraph(c))

In [8]:
H = {}

for i in range (len(G)):
    H[i] = graphs[i][0]

In [9]:
for i in range (len(G)):
    print(len(H[i]))

1


In [10]:
for i in range (len(G)):
    print(len(G[i])-len(H[i]))

43360


In [11]:
for i in range (len(G)):
    print("Check that the graph is now connected")
    print(nx.number_strongly_connected_components(H[i]))

Check that the graph is now connected
1


## Clustering

In [12]:
files = glob.glob(filepath + "/*.csv")
dataset = {}
j = 0

for filename in files:
    dataset[j] = pd.read_csv(filename, encoding="utf8")
    j += 1

In [13]:
for i in range(len(dataset)):
    print(dataset[i].head)

<bound method NDFrame.head of         id_autore agent_type     score
0           83738     voters  0.789915
1           74694     voters  0.650914
2           10739     voters  0.249978
3           90363     voters  0.888586
4           13807     voters  0.236870
...           ...        ...       ...
99996       43989     voters  0.494443
99997       68498     voters  0.626045
99998         415     voters  0.044178
99999       17596     voters  0.390634
100000      56767     voters  0.597337

[100001 rows x 3 columns]>


In [14]:
N_nodes = {}
N_edges = {}

for j in range (len(G)+1):
    N_nodes[str(j)] = [0]*8
    N_edges[str(j)] = [0]*8
    
    for i in range (len(dataset[j])):
        if dataset[j]['score'][i] < 0.1:
            N_nodes[str(j)][0] += 1
            N_edges[str(j)][0] += G[0].out_degree(dataset[j]['id_autore'][i])
        if (dataset[j]['score'][i] < 0.2) and (dataset[j]['score'][i] > 0.1):
            N_nodes[str(j)][1] += 1
            N_edges[str(j)][1] += G[0].out_degree(dataset[j]['id_autore'][i])
        if (dataset[j]['score'][i] < 0.3) and (dataset[j]['score'][i] > 0.2):
            N_nodes[str(j)][2] += 1
            N_edges[str(j)][2] += G[0].out_degree(dataset[j]['id_autore'][i])
        if (dataset[j]['score'][i] < 0.45) and (dataset[j]['score'][i] > 0.3):
            N_nodes[str(j)][3] += 1
            N_edges[str(j)][3] += G[0].out_degree(dataset[j]['id_autore'][i])
        if (dataset[j]['score'][i] < 0.55) and (dataset[j]['score'][i] > 0.45):
            N_nodes[str(j)][4] += 1
            N_edges[str(j)][4] += G[0].out_degree(dataset[j]['id_autore'][i])
        if (dataset[j]['score'][i] < 0.7) and (dataset[j]['score'][i] > 0.55):
            N_nodes[str(j)][5] += 1
            N_edges[str(j)][5] += G[0].out_degree(dataset[j]['id_autore'][i])
        if (dataset[j]['score'][i] < 0.8) and (dataset[j]['score'][i] > 0.7):
            N_nodes[str(j)][6] += 1
            N_edges[str(j)][6] += G[0].out_degree(dataset[j]['id_autore'][i])
        if (dataset[j]['score'][i] < 0.9) and (dataset[j]['score'][i] > 0.8):
            N_nodes[str(j)][7] += 1
            N_edges[str(j)][7] += G[0].out_degree(dataset[j]['id_autore'][i])

NetworkXError: nbunch is not a node or a sequence of nodes.

In [ ]:
K_mean = {}
voters_type = 8

for j in range (len(G)+1):
    K_mean[str(j)] = []
    print('\n')
    for i in range (voters_type):
        K_mean[str(j)].append(N_edges[str(j)][i]/N_nodes[str(j)][i])
        print("The ", i+1, " K_mean value is: ",K_mean[str(j)][i])